# Not working yet!

This file:
- Calculates and exports five files
- US_Sovi_Score = a sovi analysis using the entire us outputs county score and rank
- FEMA_Region_Sovi_Score = a sovi analysis by fema region outputs county score and rank
- State_Sovi_Score = a sovi analysis by state for 10 states outputs county score and rank
- county_in_state_rank = a ranking of the counties of 10 states from the us, region level, and state level analysis
- variable_contributions = net contributions of each variable in each analysis above

In [22]:
# Import modules, define directories
%run 'spss_pca.ipynb'
%run 'drop1_place.ipynb'

import pandas as pd
import geopandas as gpd
from scipy.stats import spearmanr
from pyhere import here
import numpy as np
from factor_analyzer import FactorAnalyzer

# import data_prep # testing
# from spss_pca import SPSS_PCA # testing
# from drop1_place import * # testing

pd.set_option("chained_assignment", None)

path = {
    "dscr": here("data", "scratch"),
    "drpub": here("data", "raw", "public"),
    "drpriv": here("data", "raw", "private"),
    "ddpub": here("data", "derived", "public"),
    "ddpriv": here("data", "derived", "private"),
    "rfig": here("results", "figures"),
    "roth": here("results", "other"),
    "rtab": here("results", "tables")
}

# data_prep.db1.copy() # testing
# counties['GEOID'] = counties.index.values 

skipping
skipping
skipping


In [2]:
# Import data
counties = gpd.read_file(here(path["ddpub"], "counties.gpkg"))
# counties = pd.read_csv(here(path["ddpub"], "counties.csv"))
# counties = pd.read_csv(here("data", "scratch", "counties_test.csv"))


In [3]:
# Select only the relevant columns

# Attribute name and expected influence on vulnerability
input_names = [['MEDAGE_ACS', 'pos', 'person', 'Median Age'],
               ['BLACK_ACS', 'pos', 'person', 'Pop African-American (%)'],
               ['QNATAM_ACS', 'pos', 'person', 'Pop Native American (%)'],
               ['QASIAN_ACS', 'pos', 'person', 'Pop Asian (%)'],
               ['QHISP_ACS', 'pos', 'person', 'Pop Hispanic (%)'],
               ['QAGEDEP_ACS', 'pos', 'person', 'Age Dependency (%)'],
               ['QPUNIT_ACS', 'pos', 'person', 'Persons Per Housing Unit'],
               ['PRENTER_ACS', 'pos', 'hu', 'Rental Housing (%)'],
               ['QNRRES_ACS', 'pos', 'person', 'Nursing Home Residents (%)'],
               ['QFEMALE_ACS', 'pos', 'person', 'Pop Female (%)'],
               ['QFHH_ACS', 'pos', 'hu', 'Female-Headed Households (%)'],
               ['QUNOCCHU_ACS', 'pos', 'hu', 'Vacant Housing (%)'],
               ['PERCAP_ALT', 'neg', 'person', 'Per-Capita Income'],
               ['QESL_ALT', 'pos', 'person', 'English as Second Language (%)'],
               ['QCVLUN', 'pos', 'person', 'Unemployment (%)'],
               ['QPOVTY', 'pos', 'person', 'Poverty (%)'],
               ['QMOHO', 'pos', 'hu', 'Mobile Homes (%)'],
               ['QED12LES_ALT', 'pos', 'person',
                   'Adults Completed <Grade 12 (%)'],
               ['QFEMLBR', 'pos', 'person', 'Female Employment (%)'],
               ['QEXTRCT_ALT', 'pos', 'person',
                   'Extractive Sector Employment (%)'],
               ['QSERV_ALT', 'pos', 'person', 'Service Sector Employment (%)'],
               ['QSSBEN', 'pos', 'hu', 'Social Security Income (%)'],
               ['QNOAUTO_ALT', 'pos', 'hu', 'No Automobile (%)'],
               ['QFAM', 'neg', 'person', 'Children in Married Families (%)'],
               ['QRICH200K', 'neg', 'hu', 'Annual Income >$200K (%)'],
               ['MDGRENT_ALT', 'neg', 'hu', 'Median Rent'],
               ['MHSEVAL_ALT', 'neg', 'hu', 'Median Home Value'],
               ['POPDENS', 'pos', 'person', 'Population Density']]

# Get attribute names
attr_names = [j[0] for j in input_names]
attr_names.append('GEOID')

# Select only the columns needed to compute SoVI
counties = counties[attr_names]
counties["GEOID"] = "g" + counties["GEOID"]#.astype(str)
counties['stateID'] = counties.GEOID.str.slice(0, 3, 1)
attr_names.remove('GEOID')
counties = counties.set_index(counties["GEOID"]).sort_index()
# sovi_input = sovi_input.set_index(sovi_input["GEOID"]).drop(labels = "GEOID", axis = 1)

In [4]:
counties.dtypes

MEDAGE_ACS      float64
BLACK_ACS       float64
QNATAM_ACS      float64
QASIAN_ACS      float64
QHISP_ACS       float64
QAGEDEP_ACS     float64
QPUNIT_ACS      float64
PRENTER_ACS     float64
QNRRES_ACS      float64
QFEMALE_ACS     float64
QFHH_ACS        float64
QUNOCCHU_ACS    float64
PERCAP_ALT      float64
QESL_ALT        float64
QCVLUN          float64
QPOVTY          float64
QMOHO           float64
QED12LES_ALT    float64
QFEMLBR         float64
QEXTRCT_ALT     float64
QSERV_ALT       float64
QSSBEN          float64
QNOAUTO_ALT     float64
QFAM            float64
QRICH200K       float64
MDGRENT_ALT       int64
MHSEVAL_ALT     float64
POPDENS         float64
GEOID            object
stateID          object
dtype: object

In [5]:
counties#.iloc[[14]]

,MEDAGE_ACS,BLACK_ACS,QNATAM_ACS,QASIAN_ACS,QHISP_ACS,QAGEDEP_ACS,QPUNIT_ACS,PRENTER_ACS,QNRRES_ACS,QFEMALE_ACS,...,QSERV_ALT,QSSBEN,QNOAUTO_ALT,QFAM,QRICH200K,MDGRENT_ALT,MHSEVAL_ALT,POPDENS,GEOID,stateID
GEOID,,,,,,,,,,,,,,,,,,,,,
g01001,37.0,0.179447,0.002876,0.008042,0.023997,0.187232,2.718521,0.201386,0.001942,0.513867,...,0.160870,0.292766,0.051370,0.726847,0.020116,836,137900.0,91.902357,g01001,g01
g01003,41.2,0.092356,0.005430,0.007352,0.043198,0.230693,2.484914,0.181393,0.003035,0.512787,...,0.178005,0.356091,0.030817,0.751875,0.034721,874,172900.0,115.236478,g01003,g01
g01005,38.2,0.459063,0.000910,0.002039,0.049692,0.203648,2.603099,0.264186,0.007499,0.463067,...,0.166612,0.371325,0.099756,0.467056,0.006686,577,88700.0,31.038418,g01005,g01
g01007,39.4,0.217137,0.003514,0.001098,0.018402,0.186789,2.948822,0.157066,0.003953,0.466204,...,0.134649,0.374763,0.051178,0.629812,0.003791,581,91600.0,36.547352,g01007,g01
g01009,39.1,0.012355,0.002071,0.001688,0.080848,0.211882,2.706148,0.167249,0.004020,0.504246,...,0.139538,0.370976,0.038324,0.756860,0.008701,588,115200.0,89.094574,g01009,g01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g56037,32.9,0.004352,0.001732,0.005286,0.151606,0.166781,2.613124,0.254671,0.002620,0.478218,...,0.151213,0.203145,0.032235,0.756602,0.033928,881,174600.0,4.209264,g56037,g56
g56039,36.6,0.001876,0.003892,0.005439,0.145691,0.155819,2.838133,0.223540,0.002532,0.478571,...,0.228046,0.181384,0.023611,0.820168,0.084619,1072,692700.0,5.338173,g56039,g56
g56041,34.3,0.000860,0.002197,0.000430,0.086716,0.170184,2.854463,0.212153,0.004345,0.495416,...,0.185250,0.226790,0.028142,0.794320,0.031039,634,180200.0,10.063431,g56041,g56


In [6]:
# Flipping Signs -- looks like this comes before calculating the Z-score

# To ensure that each variable contributes as expected to the final Sovi
# Index following Eric Tate (2012?) we flip the signs of the input data.
for name, sign, sample, hrname in input_names:
    if sign == 'neg':
        counties[name] = -counties[name].values
    elif sign == 'pos':
        pass
    else:
        print("problem in flipping signs")
        raise

In [7]:
# Build FEMA subRegions Dict values= state ID's
FEMA_subs = dict()
FEMA_subs['FEMA_1'] = ['g23g33g25', 'g50', 'g09', 'g44']
FEMA_subs['FEMA_2'] = ['g36', 'g34']
FEMA_subs['FEMA_3'] = ['g42', 'g10', 'g11', 'g24', 'g51', 'g54']
FEMA_subs['FEMA_4'] = ['g21', 'g47', 'g37', 'g28', 'g01', 'g13', 'g45', 'g12']
FEMA_subs['FEMA_5'] = ['g27', 'g55', 'g26', 'g17', 'g18', 'g39']
FEMA_subs['FEMA_6'] = ['g35', 'g48', 'g40', 'g05', 'g22']
FEMA_subs['FEMA_7'] = ['g31', 'g19', 'g20', 'g29']
FEMA_subs['FEMA_8'] = ['g30', 'g38', 'g56', 'g46', 'g49', 'g08']
FEMA_subs['FEMA_9'] = ['g06', 'g32', 'g04']
FEMA_subs['FEMA_10'] = ['g53', 'g41', 'g16']

In [8]:
####################################
# DataFrames to hold US, FEMA region, and state level results
####################################

# Dict to hold variable loadings
# key will be [USA, Fema_region, stateid] depending on level of analysis
varContrib = {}

# National Score
US_Sovi_Score = pd.DataFrame(index=counties.GEOID,
                             columns=['sovi', 'rank'])

# In the FEMA_Region_Sovi_Score data frame ranks are BY FEMA REGION.
# The data frame holds both the SOVI score and the county rank
# This means that there should be 10 counties with rank 1 (one for each
# FEMA Region)
FEMA_Region_Sovi_Score = pd.DataFrame(index=counties.GEOID,
                                      columns=['sovi', 'rank', 'fema_region'])

# Create New England conglomerate of states
# These are the FIPS codes for the states with the letter "g" appended
counties.loc[counties.stateID.isin(['g23', 'g33', 'g25']), 'stateID'] = 'g23g33g25'

# These are the states in the state level analysis
stateList = ['g23g33g25', 'g36', 'g51', 'g13',
             'g17', 'g48', 'g29', 'g46', 'g06', 'g16']

# In the State_Sovi_Score data frame ranks are BY STATE.
# The data frame holds both the SOVI score and the county rank
# This means that there should be 10 counties with rank 1 (one for each
# state in stateList)
State_Sovi_Score = pd.DataFrame(
    index=counties.index[counties.stateID.isin(stateList)],
    columns=['sovi', 'rank', 'state_id'])


In [9]:
State_Sovi_Score

,sovi,rank,state_id
GEOID,,,
g06001,NaN,NaN,NaN
g06003,NaN,NaN,NaN
g06005,NaN,NaN,NaN
g06007,NaN,NaN,NaN
g06009,NaN,NaN,NaN
...,...,...,...
g51800,NaN,NaN,NaN
g51810,NaN,NaN,NaN
g51820,NaN,NaN,NaN


In [10]:
print(SPSS_PCA)

<class '__main__.SPSS_PCA'>


In [36]:
?FactorAnalyzer

Init signature:
FactorAnalyzer(
    n_factors=3,
    rotation='promax',
    method='minres',
    use_smc=True,
    is_corr_matrix=False,
    bounds=(0.005, 1),
    impute='median',
    svd_method='randomized',
    rotation_kwargs=None,
)
Docstring:     
The main exploratory factor analysis class.

This class:
    (1) Fits a factor analysis model using minres, maximum likelihood,
        or principal factor extraction and returns the loading matrix
    (2) Optionally performs a rotation, with method including:

        (a) varimax (orthogonal rotation)
        (b) promax (oblique rotation)
        (c) oblimin (oblique rotation)
        (d) oblimax (orthogonal rotation)
        (e) quartimin (oblique rotation)
        (f) quartimax (orthogonal rotation)
        (g) equamax (orthogonal rotation)

Parameters
----------
n_factors : int, optional
    The number of factors to select.
    Defaults to 3.
rotation : str, optional
    The type of rotation to perform after fitting the factor analy

In [27]:
#######################
# Compute National SoVI
#######################
# compute SoVI
inputData = counties.drop(['GEOID', 'stateID'], axis=1, inplace=False)
# pca = SPSS_PCA(inputData, reduce=True, varimax=True)
# inputData_array = inputData.values  # Convert DataFrame to NumPy array
# pca = SPSS_PCA(inputData_array, reduce=True, varimax=True)

inputData_norm = (inputData - inputData.mean(axis=0)) / inputData.std(axis=0)

# fit factor analyzer with principal components and varimax rotation
fa = FactorAnalyzer(rotation="varimax", n_factors=28, method='principal')
fa.fit(inputData_norm)

# get the rotated factor pattern
loadings = pd.DataFrame(fa.loadings_, index=inputData_norm.columns, columns=[f"Factor{i+1}" for i in range(28)])

In [71]:
# calculate eigenvectors and eigenvalues
eigenvalues, eigenvectors = np.linalg.eig(fa.corr_)
eigenvalues
# sort the eigenvalues and eigenvectors in descending order
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]

# convert to dataframes
eigenvalues_df = pd.DataFrame({'Eigenvalue': eigenvalues}, index=inputData_norm.columns)
eigenvalues_df['Proportion'] = eigenvalues_df['Eigenvalue'] / eigenvalues_df['Eigenvalue'].sum()
eigenvalues_df['Cumulative Proportion'] = eigenvalues_df['Proportion'].cumsum()

# display dataframes
print("Eigenvalues:")
display(eigenvalues_df.style.format({'Eigenvalue': '{:.4f}', 'Proportion': '{:.4f}', 'Cumulative Proportion': '{:.4f}'}))

Eigenvalues:


,Eigenvalue,Proportion,Cumulative Proportion
MEDAGE_ACS,6.3940,0.2284,0.2284
BLACK_ACS,5.6512,0.2018,0.4302
QNATAM_ACS,2.8195,0.1007,0.5309
QASIAN_ACS,1.9719,0.0704,0.6013
QHISP_ACS,1.6327,0.0583,0.6596
QAGEDEP_ACS,1.3714,0.0490,0.7086
QPUNIT_ACS,1.1241,0.0401,0.7487
PRENTER_ACS,1.0827,0.0387,0.7874
QNRRES_ACS,0.8577,0.0306,0.8180
QFEMALE_ACS,0.7126,0.0254,0.8435


In [ ]:

# sovi_actual_us = pca.scores_rot.sum(1)
# sovi_actual_us = pd.DataFrame(
#     sovi_actual_us, index=sovi_input.index, columns=['sovi'])
# rank
# sovi_actual_us['rank'] = sovi_actual_us.rank(
#     method='average', ascending=False)
# US_Sovi_Score.update(sovi_actual_us)

# attrib_contribution_us = pca.weights_rot.sum(1)
# # Generate dictionary for all net loadings by variable for US
# varContrib['USA'] = zip(attr_names, attrib_contribution_us.tolist())

# # quick check of ranks max should equal number of counties in US
# try:
#     US_Sovi_Score['rank'].max() == len(sovi_input)
# except:
#     print("error in ranking check")
#     raise

# # cleanup
# del inputData
# del inputData_norm
# del sovi_actual_us
# del attrib_contribution_us


In [116]:
# pca.weights_rot
loadings
# pd.DataFrame({"sum": loadings.iloc[:, 0:8].sum(1)}).abs().sort_values("sum", ascending = False)

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,...,Factor19,Factor20,Factor21,Factor22,Factor23,Factor24,Factor25,Factor26,Factor27,Factor28
MEDAGE_ACS,-0.033345,0.830914,-0.200187,0.176309,0.104858,-0.098094,-0.026206,-0.030804,-0.026676,0.058476,...,-0.015613,-0.031978,-0.014546,-0.173847,-0.014154,-0.001801,-0.296615,-0.002005,-0.046381,-0.006789
BLACK_ACS,0.001198,-0.112421,-0.122691,0.019342,-0.016911,-0.169548,0.012244,0.023593,0.025714,-0.002243,...,0.469759,0.038710,0.062627,0.011070,-0.004820,0.008316,0.004969,-0.003103,-0.005080,0.000688
QNATAM_ACS,0.048066,-0.083887,-0.028966,0.066748,-0.026975,0.970246,0.050691,-0.037397,-0.022246,0.049779,...,-0.021942,-0.012932,-0.003949,0.014583,0.004018,0.001130,0.003800,0.002066,-0.001580,0.000619
QASIAN_ACS,-0.414006,-0.158809,0.142751,-0.045950,-0.020064,-0.002695,-0.010404,-0.023929,0.110754,-0.071581,...,0.001121,-0.015201,-0.013508,0.097897,-0.017417,0.006119,0.001252,0.000616,-0.001934,-0.000180
QHISP_ACS,-0.022695,-0.101520,0.949206,0.017891,-0.065510,-0.039829,0.092888,-0.037782,0.017950,0.093235,...,-0.040799,0.010446,0.021286,0.031380,-0.004911,-0.208410,0.003776,0.002988,0.003744,0.001687
QAGEDEP_ACS,0.187899,0.877111,-0.032115,0.114690,0.208172,0.006602,0.032711,0.148354,-0.017755,0.156731,...,0.011884,-0.025578,-0.078108,-0.015424,0.022489,0.005101,0.168918,-0.012285,-0.136367,0.000921
QPUNIT_ACS,-0.001586,-0.393394,0.331016,-0.055046,-0.153511,0.216579,-0.020177,0.019394,-0.026950,-0.029689,...,0.016837,0.069405,0.103062,-0.072238,-0.018391,0.003826,0.007750,0.008286,-0.000713,0.001732
PRENTER_ACS,-0.088567,-0.422108,0.141092,-0.289111,0.008204,0.031868,0.093256,0.081898,0.166826,-0.017087,...,0.000925,-0.007946,-0.018993,0.718775,-0.013458,0.002748,0.010528,0.001235,-0.004539,0.000604
QNRRES_ACS,0.136088,0.228814,-0.091657,-0.040189,0.939473,-0.031033,0.012236,-0.041646,-0.007025,0.083522,...,-0.001985,-0.001357,-0.052072,0.003050,0.016045,-0.000180,-0.003429,-0.001846,0.000551,-0.000525
QFEMALE_ACS,-0.038771,0.071659,-0.069243,-0.098316,-0.038873,-0.038361,-0.087419,0.959674,0.037331,-0.082284,...,0.002658,-0.010497,-0.060315,0.036460,-0.001310,-0.000918,0.002591,0.002360,-0.000926,0.000332


In [13]:
counties.to_csv(here(path["dscr"],'counties_test.csv'))  

In [114]:
?pd.DataFrame.sort_values

Signature:
pd.DataFrame.sort_values(
    self,
    by,
    axis: 'Axis' = 0,
    ascending=True,
    inplace: 'bool' = False,
    kind: 'str' = 'quicksort',
    na_position: 'str' = 'last',
    ignore_index: 'bool' = False,
    key: 'ValueKeyFunc' = None,
)
Docstring:
Sort by the values along either axis.

Parameters
----------
        by : str or list of str
            Name or list of names to sort by.

            - if `axis` is 0 or `'index'` then `by` may contain index
              levels and/or column labels.
            - if `axis` is 1 or `'columns'` then `by` may contain column
              levels and/or index labels.
axis : {0 or 'index', 1 or 'columns'}, default 0
     Axis to be sorted.
ascending : bool or list of bool, default True
     Sort ascending vs. descending. Specify list for multiple sort
     orders.  If this is a list of bools, must match the length of
     the by.
inplace : bool, default False
     If True, perform operation in-place.
kind : {'quicksort', 'me

In [15]:



######################
# FEMA REGION SOVI
######################
for i in FEMA_subs:

    # Subset FEMA subregion
    FEMARegionData = US_All[US_All['stateID'].isin(FEMA_subs[i])]

    # compute SoVI
    inputData = FEMARegionData.drop(
        ['Geo_FIPS', 'stateID'], axis=1, inplace=False)
    # pca = SPSS_PCA(inputData, reduce=True, varimax=True)
    
    #NEW
    inputData_array = inputData.values  # Convert DataFrame to NumPy array
    pca = SPSS_PCA(inputData_array, reduce=True, varimax=True)
    
    sovi_actual_fema = pca.scores_rot.sum(1)

    # load into df for merge
    sovi_actual_fema = pd.DataFrame(
        sovi_actual_fema, index=FEMARegionData.index, columns=['sovi'])
    # add fema region to df
    sovi_actual_fema['fema_region'] = i
    # rank
    sovi_actual_fema['rank'] = sovi_actual_fema['sovi'].rank(
        method='average', ascending=False)

    FEMA_Region_Sovi_Score.update(sovi_actual_fema)

    attrib_contribution_fema = pca.weights_rot.sum(1)

    # Write attribute contribution output
    # Generate dictionary for all net loadings by variable and region
    varContrib[i] = zip(attr_names, attrib_contribution_fema.tolist())

# cleanup
del FEMARegionData
del inputData
del sovi_actual_fema
del attrib_contribution_fema

#############################################
# State Analysis
#############################################
for st in stateList:
    # Subset FEMA subregion
    stateData = US_All[US_All.stateID == st]

    # compute SoVI
    inputData = stateData.drop(['Geo_FIPS', 'stateID'], axis=1, inplace=False)
    # pca = SPSS_PCA(inputData, reduce=True, varimax=True)
    
    # NEW:
    inputData_array = inputData.values  # Convert DataFrame to NumPy array
    pca = SPSS_PCA(inputData_array, reduce=True, varimax=True)
    
    sovi_actual = pca.scores_rot.sum(1)
    sovi_actual = pd.DataFrame(
        sovi_actual, index=stateData.index, columns=['sovi'])
    sovi_actual['state_id'] = st
    # rank w/in state
    sovi_actual['rank'] = sovi_actual['sovi'].rank(
        method='average', ascending=False)
    State_Sovi_Score.update(sovi_actual)
    attrib_contribution = pca.weights_rot.sum(1)
    varContrib[st] = zip(attr_names, attrib_contribution.tolist())

# cleanup
del stateData
del inputData
del sovi_actual
del attrib_contribution

###################################################
# Make Var Contributions Data Frame
###################################################
variable_contributions = pd.DataFrame(index=attr_names)
# for area in varContrib.iterkeys():
for area in varContrib.keys():
    variable_contributions[area] = [x for i, x in varContrib[area]]

##########################################################################
# Ranks w/ Geographic Extent
# For each county rank within state for US, state, and fema_region sovis
##########################################################################

county_in_state_rank = pd.DataFrame(index=State_Sovi_Score.index,
                                    columns=['state_sovi_rank', 'fema_region_sovi_rank', 'us_sovi_rank'])

for st in stateList:
    if st == 'g23g33g25':
        # get all counties in the three NE states and rank for us
        st_cty_scores = US_Sovi_Score.loc[
            ['g23' in s for s in US_Sovi_Score.index], 'sovi']
        st_cty_scores = st_cty_scores.append(
            US_Sovi_Score.loc[['g33' in s for s in US_Sovi_Score.index], 'sovi'])
        st_cty_scores = st_cty_scores.append(
            US_Sovi_Score.loc[['g25' in s for s in US_Sovi_Score.index], 'sovi'])

        county_in_state_rank.loc[st_cty_scores.index, 'us_sovi_rank'] = st_cty_scores.rank(method='average', ascending=False)

        # get all counties in state and rank for fema region
        st_cty_scores = FEMA_Region_Sovi_Score.loc[
            ['g23' in s for s in FEMA_Region_Sovi_Score.index], 'sovi']
        st_cty_scores = st_cty_scores.append(FEMA_Region_Sovi_Score.loc[
                             ['g33' in s for s in FEMA_Region_Sovi_Score.index], 'sovi'])
        st_cty_scores = st_cty_scores.append(FEMA_Region_Sovi_Score.loc[
                             ['g25' in s for s in FEMA_Region_Sovi_Score.index], 'sovi'])

        county_in_state_rank.loc[st_cty_scores.index, 'fema_region_sovi_rank'] = st_cty_scores.rank(method='average', ascending=False)

        # county rank in state only sovi
        county_in_state_rank.loc[st_cty_scores.index, 'state_sovi_rank'] = State_Sovi_Score.loc[State_Sovi_Score['state_id'] == 'g23g33g25', 'rank']

    else:
        st_cty_scores = US_Sovi_Score.loc[[st in s for s in US_Sovi_Score.index], 'sovi']
        county_in_state_rank.loc[st_cty_scores.index, 'us_sovi_rank'] = st_cty_scores.rank(method='average', ascending=False)
        # get all counties in state and rank for fema region
        st_cty_scores = FEMA_Region_Sovi_Score.loc[[st in s for s in FEMA_Region_Sovi_Score.index], 'sovi']
        county_in_state_rank.loc[st_cty_scores.index, 'fema_region_sovi_rank'] = st_cty_scores.rank(method='average', ascending=False)

        # county rank in state only sovi
        st_cty_scores = State_Sovi_Score.loc[State_Sovi_Score['state_id'] == st, 'rank']
        county_in_state_rank.loc[st_cty_scores.index, 'state_sovi_rank'] = st_cty_scores

#####################################################
# Drop 1 Variable
#####################################################
USvarRanks = variable_contributions.USA.sort_values()
dropLevels = USvarRanks.index

#build multindex
geoLevels = US_All.Geo_FIPS
geoLabels = []
for _ in range(len(dropLevels)):
    geoLabels.extend(range(len(geoLevels)))
dropLabels = np.repeat(range(len(dropLevels)), len(geoLevels))

US_Drop1_Multi_Index = pd.MultiIndex(levels=[dropLevels, geoLevels],
                                    codes=[dropLabels, geoLabels], # was labels but was getting error, looked at documentation and switched to a parameter that sounded similar
                                    names=['DroppedVar', 'Geo_FIPS'])

US_Drop1_NetContrib = pd.DataFrame(index=dropLevels, columns=dropLevels)

US_SoVI_Drop1_Score = pd.DataFrame(index=US_Drop1_Multi_Index, columns=['sovi'])


for j in dropLevels:
    US_dropj = US_All.drop([j, 'Geo_FIPS', 'stateID'], axis=1, inplace=False)
    # pca = SPSS_PCA(US_dropj, reduce=True, varimax=True)
    # NEW
    US_dropj_array = US_dropj.values  # Convert DataFrame to NumPy array
    pca = SPSS_PCA(US_dropj_array, reduce=True, varimax=True)
    
    sovi_actual = pca.scores_rot.sum(1)
    sovi_actual = pd.DataFrame(sovi_actual, index=geoLevels, columns=['sovi'])
    US_SoVI_Drop1_Score.loc[j, 'sovi'] = sovi_actual.values
    attrib_contribution = pd.DataFrame(data=pca.weights_rot.sum(1), index=US_dropj.columns)

    attrib_contribution = attrib_contribution.transpose()
    attrib_contribution.index = [j]
    US_Drop1_NetContrib.loc[attrib_contribution.columns,j] = attrib_contribution.loc[j, :]


# sort by rank order
US_rank_order=abs(variable_contributions.USA).rank(method='average',ascending=False).sort_values().index # original rank order
US_Drop1_NetContrib=US_Drop1_NetContrib.loc[US_rank_order] # sort rows
US_Drop1_NetContrib=US_Drop1_NetContrib.loc[:,US_rank_order] # sort columns

# ranked version of the drop 1 variable table
US_Drop1_NetContrib_ranks=US_Drop1_NetContrib.copy()
US_Drop1_NetContrib_ranks=US_Drop1_NetContrib_ranks.apply(lambda x: abs(x).rank(method='average',ascending=False)) # convert absolute scores to ranks
US_Drop1_NetContrib_ranks=US_Drop1_NetContrib_ranks.loc[US_rank_order] # sort rows
US_Drop1_NetContrib_ranks=US_Drop1_NetContrib_ranks.loc[:,US_rank_order] # sort columns

######################
# CORRELATIONS
######################
state_corrs = pd.DataFrame(index = stateList, columns = ['spearman_r_st_fema', 'pvalue_st_fema', 'spearman_r_st_us', 'pvalue_st_us'])
for st in stateList:
  if st == 'g23g33g25':
    multi_state_data_tmp = county_in_state_rank.loc[['g23' in s for s in county_in_state_rank.index], ]
    multi_state_data_tmp = multi_state_data_tmp.append(
      county_in_state_rank.loc[['g25' in s for s in county_in_state_rank.index], ])
    multi_state_data_tmp = multi_state_data_tmp.append(
      county_in_state_rank.loc[['g33' in s for s in county_in_state_rank.index], ])
    st_fema_spearman = spearmanr(multi_state_data_tmp[['state_sovi_rank', 'fema_region_sovi_rank']])
    st_us_spearman = spearmanr(multi_state_data_tmp[['state_sovi_rank', 'us_sovi_rank']])
    state_corrs.loc['g23g33g25', ] = [st_fema_spearman[0], st_fema_spearman[1], st_us_spearman[0], st_us_spearman[1]]
  else:
    st_fema_spearman = spearmanr(county_in_state_rank.loc[[st in s for s in county_in_state_rank.index], ['state_sovi_rank', 'fema_region_sovi_rank']])
    st_us_spearman = spearmanr(county_in_state_rank.loc[[st in s for s in county_in_state_rank.index], ['state_sovi_rank', 'us_sovi_rank']])
    state_corrs.loc[st, ] = [st_fema_spearman[0], st_fema_spearman[1], st_us_spearman[0], st_us_spearman[1]]

################
# DROP ONE PLACE
################

# df containing county names - no need for the geometr#
# county_names=pd.DataFrame(gpd.read_file(os.path.join(s#path,'USA_Counties_500k.shp('../data/
county_names=pd.read_csv(os.path.join(inputs,'county_names.csv'),index_col=0,encoding='latin-1')

# ##### State (California)
print('\nDrop One Place: State\n')
# spearman rank correlations
ca_cors=dropCors(US_All,State_Sovi_Score,'g06')

# drop run with minimum rank correlation
cad=ca_cors[ca_cors==min(ca_cors)].index.values[0]

# rank change table with minimum rank correlation
ca_rchg=rankChgTable(inputs=US_All,scores=State_Sovi_Score,obs_names=county_names,subset='g06',drop=cad,cor=True,top=10)

# rank quantile moves with minimum rank correlation
ca_quint_moves=rankQuantileMoves(inputs=US_All,scores=State_Sovi_Score,subset='g06',drop=cad)

# ##### FEMA 9: California and surrounding states (includes Hawaii)
print('Drop One Place: FEMA\n')

f9_cors=dropCors(US_All,FEMA_Region_Sovi_Score,'FEMA_9')

# obs that decreases the correlation most when dropped
f9cd=f9_cors[f9_cors==min(f9_cors)].index.values[0]

f9_rchg=rankChgTable(inputs=US_All,scores=FEMA_Region_Sovi_Score,obs_names=county_names,subset='FEMA_9',drop=f9cd,cor=True,top=10)

# rank quantile moves
f9_quint_moves=rankQuantileMoves(inputs=US_All,scores=FEMA_Region_Sovi_Score,subset='FEMA_9',drop=f9cd)

# ### Full USA
print('Drop One Place: USA\n')

us_cors=dropCors(US_All,US_Sovi_Score)

# obs that decreases the correlation most when dropped
uscd=us_cors[us_cors==min(us_cors)].index.values[0]

us_rchg=rankChgTable(inputs=US_All,scores=US_Sovi_Score,obs_names=county_names,drop=uscd,cor=True,top=10)

# rank quantile moves
us_quint_moves=rankQuantileMoves(inputs=US_All,scores=US_Sovi_Score,drop=uscd)
print('\n')

# cleanup
del multi_state_data_tmp
#####################################################
# OUTPUT TABLES
#####################################################
US_Sovi_Score.to_csv(os.path.join(outputs, 'US_Sovi_Score.csv'))

# In the FEMA_Region_Sovi_Score data frame ranks are BY FEMA REGION.
# The data frame holds both the SOVI score and the county rank
# This means that there should be 10 counties with rank 1 (one for each
# FEMA Region)
FEMA_Region_Sovi_Score.to_csv(os.path.join(
    outputs, 'FEMA_Region_Sovi_Score.csv'))

# In the State_Sovi_Score data frame ranks are BY STATE.
# The data frame holds both the SOVI score and the county rank
# This means that there should be 10 counties with rank 1 (one for each
# state in stateList)
State_Sovi_Score.to_csv(os.path.join(
    outputs, 'State_Sovi_Score.csv'))

# County rank within state for US, state, and fema_region sovis
county_in_state_rank.to_csv(os.path.join(
    outputs, 'County_in_State_Rank.csv'))

# Variable contributions for sovis at all geographic extents
variable_contributions.to_csv(os.path.join(
    outputs, 'variable_contributions.csv'))

# Net contribution of variables after dropping a variable
US_Drop1_NetContrib.to_csv(os.path.join(
    outputs, 'US_Drop1_NetContrib_raw.csv'))

# rank of variables after dropping a variable
US_Drop1_NetContrib_ranks.to_csv(os.path.join(
    outputs, 'US_Drop1_NetContrib_ranks.csv'))

# Correlation of ranks
state_corrs.to_csv(os.path.join(
    outputs, 'state_fema_us_rank_correlations.csv'))

# Drop 1 place
ca_rchg.to_csv(os.path.join(outputs,'drop1_place_state_rank_change_ca.csv'))
ca_quint_moves.to_csv(os.path.join(outputs,'drop1_place_state_quint_moves_ca.csv'))
f9_rchg.to_csv(os.path.join(outputs,'drop1_place_fema_rank_change_fema9.csv'))
f9_quint_moves.to_csv(os.path.join(outputs,'drop1_place_fema_quint_moves_fema9.csv'))
us_rchg.to_csv(os.path.join(outputs,'drop1_place_usa_rank_change.csv'))
us_quint_moves.to_csv(os.path.join(outputs,'drop1_place_usa_quint_moves.csv'))


NameError: name 'US_All' is not defined